In [ ]:
import anthropic

client = anthropic.Anthropic()

In [10]:
def get_answer_prompt(problem:str) -> str:
    prompt = """Write Python code to solve this problem:

    {problem}
    
    Don't forget the imports.""".format(problem=problem)
    return prompt

In [11]:
from datasets import load_dataset

h_eval = load_dataset('openai/openai_humaneval')["test"]

In [17]:
from helper_functions import extract_python_code, max_index_of_min_value
from test_execute import unsafe_execute
from notebook_helper import create_notebook

def generate_notebook_to_problem(problem, save_path):
    messages = [{"role":"user", "content":[{"type":"text", "text":get_answer_prompt(problem["prompt"])}]}]
    time_list = []
    answer_list = []
    for i in range(5):
        answer = client.messages.create(model="claude-3-7-sonnet-20250219",
            max_tokens=1000, temperature=0, messages=messages)
        answer_list.append(answer.content[0].text)
        text = extract_python_code(answer.content[0].text)
        messages.append({"role":"assistant", "content":[{"type":"text", "text":text}]})    
        status, time_taken = unsafe_execute(problem, text)
        if status == "passed":
            print(time_taken)
            messages.append({"role":"user", "content":[{"type":"text", "text":"Write more optimized code, which objective is to make the code faster. Explain your optimizations."}]})
        else:
            messages.append({"role":"user", "content":[{"type":"text", "text":f"There was this error: {status}. Write better code. Explain your optimizations."}]})
        time_list.append(time_taken)
    max_time_index = max_index_of_min_value(time_list)
    first_answer = answer_list[0]
    modified_answer = answer_list[max_time_index]
    print(modified_answer)
    create_notebook(problem, first_answer, modified_answer, save_path)

In [ ]:
for i in range(5):
    problem = h_eval[i]
    save_path = "optimized_problems/problem_{i}.ipynb".format(i=i)
    generate_notebook_to_problem(problem, save_path)

0.0009253025054931641
0.0
0.0
